<a href="https://colab.research.google.com/github/iordach1/GEOL572-GRL/blob/master/GRL_Model_w_calibration(transmissivityadded).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>FloPy Regional Model Development</center>
The following script develops a regional MODFLOW model using FloPy and associated packages. The study domain is the Mahomet Aquifer in Illinois, particulary in the heavily irrigated region of Mason County. 

Note that you may not always have to compile MODFLOW, and you only have initialize code (#1) and import large files (#2) the first time you run the code in a session. Most of the time, you will only have to run #3 onward. 

# 0. Compile MODFLOW (DO NOT RUN UNLESS NECESSARY)


When using Google CoLab, you cannot simply use an existing MODFLOW executable, but have to compile a a new one for use in this environment. I have already compiled MODFLOW, but am currently unsure if it will work for everybody. Please do not run the following code unless you get an error that the MODFLOW file was not found (will occur when running the model). 

In [0]:
#!pip install https://github.com/modflowpy/pymake/zipball/master # install of the pymake package to compile MODFLOW

In [0]:
# Code to compile the MODFLOW executable

#import pymake

#def make_mf2005():
#    pymake.build_apps('mf2005')


#if __name__ == "__main__":
#    make_mf2005()





#import pymake

#def make_mfnwt():
#    pymake.build_apps('mfnwt')

#if __name__ == "__main__":
#    make_mfnwt()


# 1 Initializing the code


These few code blocks only have to be run at the beginning of the script or if a runtime connection is lost. See details for each below. 

## 1A. Install needed packages
There are three packages that must be installed in the CoLab environment to run this script. These include 

"pyproj", which allows for the conversion from WGS84 coordinates (as obtained from Google Earth), which are in lat/long decimal degrees, to Illimap coordinates, a specialized projection using Lambert Conformal Conic that is optimized for developing a model grid in the state of Illinois. 

"flopy" creates modflow executables, runs the model, and allows for manipulation of results.

"rasterio" allows for the reading of raster files, and is closely tied to the development of "flopy" input files


In [0]:
#Get the EEG, the BP monitor, and the AVV.
!pip install pyproj
!pip install flopy
!pip install rasterio
!pip install pykrige
!pip install metpy
!apt-get -qq install python-cartopy python3-cartopy

## 1B. Import packages

In [0]:
#And get the machine that goes 'ping!'.
import flopy # import flopy to develop, run, and analyze the model
from flopy.utils import Raster # plot rasters from flopy
import matplotlib as mp
import pandas as pd
import pyproj # change between WGS84 and Illimap coordinates
import rasterio  # import rasters
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np 
import json
import math
import cartopy
import cartopy.crs as ccrs # import projections
import cartopy.feature as cf # import features
from pykrige.uk import UniversalKriging
import pylab # using as a plotting library for spatial data, make contours
from metpy.plots import USCOUNTIES

#And get the most expensive machine - in case the Administrator comes.
# the following code authorizes you to access files on Google Drive
from google.colab import drive, files, auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials


## 1C. Authenticate with Google
This will allow you to access shared files on my Google Drive or your own. 

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## 1D. Read the MODFLOW executable from Google Drive

Here you are going to download the MODFLOW executable from Daniel's Google Drive. This will show up in your temporary working directory for use later in the code. <b>We need to confirm whether this will work in CoLab, or if you need to compile a new version of MODFLOW each time.</b>

Note that this makes use of a key on your Google Drive sharable link for the file. This generally takes the form of: https://drive.google.com/file/d/**key**/view?usp=sharing, in this case, key = 1SwqsSCvyrnrCiopeEp52LBcZYNcgBT9V

In [0]:
downloaded = drive.CreateFile({'id':"1SwqsSCvyrnrCiopeEp52LBcZYNcgBT9V"}) # This is the key that is defined in the shareable link of a file on Google Drive
downloaded.GetContentFile('mf2005') # get the content from Google Drive and name it (let's stick with mf2005 here)

#downloaded = drive.CreateFile({'id':"1iThxRoGVhofLPd8HOF7Vn0awRVrXZcVb"}) # This is the key that is defined in the shareable link of a file on Google Drive
#downloaded.GetContentFile('mfnwt') # get the content from Google Drive and name it (let's stick with mf2005 here)

Now we need to copy the executable into a location where we can later run it. Note that the "!" nomenclature is a shell command, beyond the scope of this course to learn that!

In [0]:
!cp mf2005 /usr/local/bin
!chmod 755 /usr/local/bin/mf2005

#!cp mfnwt /usr/local/bin
#!chmod 755 /usr/local/bin/mfnwt

# 2 Import the big stuff


We don't want to repeatedly import large files for fear of Google revoking our CoLab priveleges for a few hours. Once or twice isn't a big deal, but repeatedly might be an issue. Let Daniel know if this happens- there are ways to reduce our file sizes. 

For now, the "big stuff" includes a DEM of Illinois and the river excel file we worked with last time. 

## 2A. Import the DEM

In [0]:
# first import the land surface .tif from Google Drive
downloaded = drive.CreateFile({'id':"1389l8sgQ8-tsmIZuZosaqvbqpHY40n6l"}) # ft above msl, works best for most groups, spiky in Mason County 
downloaded.GetContentFile('landsurface_el.tif')

# first import the bedrock elevation .tif from Google Drive
downloaded = drive.CreateFile({'id':"1EZgZDjjILzvRzvY9nf0Qp0NHmspRq4kP"})   
downloaded.GetContentFile('bedrock_el.tif')

# read in percent thickness of coarse grain for each model layer
downloaded = drive.CreateFile({'id':"18Kw3O6qCzIJ2L6KrVnRPIhea_F8VwyWn"})   
downloaded.GetContentFile('percentl1.tif')

downloaded = drive.CreateFile({'id':"1oZinFPKrGY-FXoE7Zu0okFpAAOe_bwau"})   
downloaded.GetContentFile('percentl2.tif')

downloaded = drive.CreateFile({'id':"1FqVEr4m_ElUyEZeyfnCMwVGDfUqavJZH"})   
downloaded.GetContentFile('percentl3.tif')

downloaded = drive.CreateFile({'id':"1KiHS9TLSP1GAVTjaaJZS4BAwF6gnUeDu"})   
downloaded.GetContentFile('percentl4.tif')

downloaded = drive.CreateFile({'id':"1Z-9EyaAK1NKnRHAlnyGYkI3suvBFC2I6"})   
downloaded.GetContentFile('percentl5.tif')

downloaded = drive.CreateFile({'id':"1pcB9aJpJGfkXOKz10rhs6MpWkQL1_dqr"})   
downloaded.GetContentFile('percentl6.tif')

downloaded = drive.CreateFile({'id':"1Fnh0HIKbUj7pEtlsUKR_Sr7WwfYzWul5"})   
downloaded.GetContentFile('percentl7.tif')

downloaded = drive.CreateFile({'id':"106JacgpwSA3wVAGcBIzGdc8rDVUB6dh7"})   
downloaded.GetContentFile('percentl8.tif')

downloaded = drive.CreateFile({'id':"1WJjhVJ_KSBhZDrgzY3YteNjxaz5nxBid"})   
downloaded.GetContentFile('percentl9.tif')

## 2B. Import the river file

In [0]:
# first import the Excel file from Google Drive
downloaded = drive.CreateFile({'id':"1JsAiGG4RvcfYrQtfgXRW9ZVfAkQ1yRVu"})
downloaded.GetContentFile('rivers_625.csv')

# 3 Create MODFLOW  packages

## 3A. Model Set-up

### 3Ai. Define the Model Domain

In [0]:
#----------------------------------------------------------------------------
'''GRL'''
sw_lat =  41.2 #southwest latitude
sw_long = -90.5 #southwest longitude
ne_lat =  41.9 #northeast latitude
ne_long = -89.2 #northeast longitude

illimap = {'proj': 'lcc', # Lambert Conformal Conic
     'ellps': 'clrk66',
     'lon_0': -89.5,
     'lat_0': 33,
     'lat_1': 33,
     'lat_2': 45,
     'x_0': 2999994*0.3048,
     'y_0': 0}

prj = pyproj.Proj(illimap)

wgs84 = pyproj.Proj("epsg:4326")

nex, ney = pyproj.transform(wgs84,illimap,ne_lat,ne_long)
swx, swy = pyproj.transform(wgs84,illimap,sw_lat,sw_long)

nex, ney = round(nex/0.3048,-4), round(ney/0.3048,-4)
swx, swy = round(swx/0.3048,-4), round(swy/0.3048,-4)

### 3Aii. Define spatial and temporal discretization

In [0]:
# Assign Discretization variables
Lx = nex-swx # Width of the model domain
Ly = ney-swy # Height of the model domain
nlay = 10 # Number of model layers
dx = 2000 
dy = 2000
nrow = int(Ly/dy) # Number of rows
ncol = int(Lx/dx) # Number of columns

nper = 1 #specify number of stress periods
steady = [True] #specify if stress period is transient or steady-state



### 3Aiii. Define river elevations



In [0]:
# import stage, lambert x, lambert y
dfriv = pd.read_csv('rivers_625.csv')

# trim dataframe with river information to the model domain
dfriv = dfriv.loc[dfriv['lamx']<nex]
dfriv = dfriv.loc[dfriv['lamy']<ney]
dfriv = dfriv.loc[dfriv['lamx']>swx]
dfriv = dfriv.loc[dfriv['lamy']>swy]

# assign all rivers to the upper layer
dfriv['lay'] = 0
# convert lamx to column and lamy to row
dfriv['row'] = np.trunc((ney-dfriv['lamy'])/dy)
dfriv['col'] = np.trunc((dfriv['lamx']-swx)/dx)
# define the river stage
dfriv['stage'] = dfriv['rvr_stg']
#define the conductance
dfriv['cond'] = 5000. # ft^2/d
# define the river bottom
dfriv['bot'] = dfriv['stage']-3
# drop unneeded files
dfriv = dfriv.drop(['STR_ORD_MI','STR_ORD_MA','SUM_LENGTH','rvr_stg','lamx','lamy'],axis=1)

dfriv = dfriv.groupby(['lay','row','col'],as_index=False).mean()

### 3Aiv. Define top and bottom elevations

In [0]:
# Now load the raster using FloPy's built in Raster toolbox
illinoisdem = Raster.load("landsurface_el.tif")
bedrock = Raster.load("bedrock_el.tif")

# Crop the dEM to the model domain
illinoisdem.crop([(swx,swy),(swx,ney),(nex,ney),(nex,swy)])
bedrock.crop([(swx,swy),(swx,ney),(nex,ney),(nex,swy)])
# Define centroid of the southwestern most cell
startx = swx+dx/2 
starty = swy+dx/2
# Calculate the x and y coordinates for the centroid of each cell 
xc = np.arange(swx+dx/2,nex+dx/2,dx) 
yc = np.arange(swy+dy/2,ney+dy/2,dy)
# Create a grid of the x coordinate of each centroid and the y coordinate
xarr, yarr = np.meshgrid(xc,yc)
# resample the topo raster to the grid of centroids of the model
topgrid = illinoisdem.resample_to_grid(xarr,yarr,1,method='nearest') 
bedrock = bedrock.resample_to_grid(xarr,yarr,1,method='nearest')

# We just built our top elevation upside down, let's flip it
topgrid = np.flipud(topgrid) 
bedrockgrid = np.flipud(bedrock)   

# the grid does not extend past the boundaries of Illinois
# the following code is for groups working in the western part of the state
maxrow = topgrid.shape[0]
maxcol = topgrid.shape[1]

# Create ibound as array of ints (1), indicating all cells are active
# inactivate cells west of the Mississippi River that were originally not present
# note that because inactive cells would overlap with the river boundaries, this code pushes inactive cells to the west a bit. Adjust per your model domain
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)

for row in np.arange(maxrow,0,-1):
  counter = 0
  for col in np.arange(maxcol,0,-1):
    if topgrid[row-1,col-1] <= 0:
      counter = counter+1
      topgrid[row-1,col-1] = topgrid[row-1,col]
      bedrockgrid[row-1,col-1] = bedrockgrid[row-1,col]
      if counter>=4:
        ibound[:,row-1,col-1] = 0




# Set the top of Layer 1 = River Elevation
for index, row in dfriv.iterrows():  
    topgrid[int(row['row']),int(row['col'])]=row['stage'] 
    #print(topgrid[int(row['row']),int(row['col'])],row['bot'])  

# make sure that all layers are at least 9 ft thick
diff = topgrid-bedrockgrid

diff[diff<=9.0] = 9.0

# each layer is the same thickness, we need 9 glacial layers
laythick = diff/9

#calculate bottom of each layer
lay1bot = topgrid-laythick
lay2bot = topgrid-2*laythick
lay3bot = topgrid-3*laythick
lay4bot = topgrid-4*laythick
lay5bot = topgrid-5*laythick
lay6bot = topgrid-6*laythick
lay7bot = topgrid-7*laythick
lay8bot = topgrid-8*laythick
lay9bot = topgrid-9*laythick
lay10bot = lay9bot-50.

botgrids = [lay1bot,lay2bot,lay3bot,lay4bot,lay5bot,lay6bot,lay7bot,lay8bot,lay9bot,lay10bot]

### 3Av. Assign hydraulic conductivity

In [0]:
# assign hydraulic conductivity in ft/day
kc = 240 # predominantly coarse
kf = .0013 # predominantly fine
kb = 20 # bedrock

# determine how to assign hydraulic conductivity
threshold = 52 # anything above this will be assigned kc and below kf

def kloader(rastername, kc, kf, threshold):
  percent = Raster.load(rastername) # load raster
  percent.crop([(swx,swy),(swx,ney),(nex,ney),(nex,swy)]) # crop array
  percentgrid = percent.resample_to_grid(xarr,yarr,1,method='nearest') # resample to model grid
  percentgrid = np.flipud(percentgrid) # flip the grid
  maxrow = percentgrid.shape[0]
  maxcol = percentgrid.shape[1]
  for row in np.arange(maxrow,0,-1):
    for col in np.arange(maxcol,0,-1):
      if percentgrid[row-1,col-1] < -10:
        percentgrid[row-1,col-1] = percentgrid[row-1,col]
  percentgrid[percentgrid>=threshold] = kc # assign coarse k value
  percentgrid[percentgrid<threshold] = kf # assign fine k value
  return percentgrid

kl1 = kloader('percentl1.tif',kc,kf,threshold)
kl2 = kloader('percentl2.tif',kc,kf,threshold)
kl3 = kloader('percentl3.tif',kc,kf,threshold)
kl4 = kloader('percentl4.tif',kc,kf,threshold)
kl5 = kloader('percentl5.tif',kc,kf,threshold)
kl6 = kloader('percentl6.tif',kc,kf,threshold)
kl7 = kloader('percentl7.tif',kc,kf,threshold)
kl8 = kloader('percentl8.tif',kc,kf,threshold)
kl9 = kloader('percentl9.tif',kc,kf,threshold)
kl10 = kl9-kl9+kb

khlayers = [kl1,kl2,kl3,kl4,kl5,kl6,kl7,kl8,kl9,kl10]
kvlayers=np.divide(khlayers,10.)

In [0]:
# Creating calculations for Transmissivity - ft^2/day (same as that which was derived by Anthony as part of milestone 1)

# because we know that each model layer is the same thickness [ft] and K [ft/day] is calculated above: 

tl1 = (kl1*laythick)
tl2 = (kl2*laythick)
tl3 = (kl3*laythick)
tl4 = (kl4*laythick)
tl5 = (kl5*laythick)
tl6 = (kl6*laythick)
tl7 = (kl7*laythick)
tl8 = (kl8*laythick)
tl9 = (kl9*laythick)
tl10 = (kl10*laythick)


Total = tl1+tl2+tl3+tl4+tl5+tl6+tl7+tl8+tl9+tl10

Tmax = np.max(Total)
Tmin = np.min(Total)


## 3B. Create the MODFLOW model object 

In [0]:
# Create a MODFLOW model object and run with MODFLOW 2005.
modelname = "my_model" # name the model
m = flopy.modflow.Modflow(modelname, version = 'mf2005', exe_name = 'mf2005') # create model object m

## 3C. Append the discretization package to the model object

In [0]:
# length and time are feet (1) and days (4).
# See https://water.usgs.gov/ogw/modflow/MODFLOW-2005-Guide/index.html?dis.htm 
dis = flopy.modflow.ModflowDis(model=m, nlay=nlay, nrow=nrow, ncol=ncol, 
                               delr=dx, delc=dy, top=topgrid, botm=botgrids, 
                               itmuni = 4, lenuni = 1, 
                               nper=nper, steady=steady)

## 3D. Basic Package

In [0]:
# Create ibound as array of ints (1), indicating all cells are active
#ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)

#Create flopy bas object
bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=topgrid)


## 3E. LPF Package

In [0]:
#define layer type as convertible (1), must be an integer
#for more information, see https://water.usgs.gov/ogw/modflow/MODFLOW-2005-Guide/index.html?dis.htm
laytyp = 0*np.ones((nlay,), dtype=np.int32)

# create the LPF object
lpf = flopy.modflow.ModflowLpf(model=m, hk=khlayers, vka=kvlayers, laytyp=laytyp, ipakcb=1)


## 3F. Boundary Conditions

### 3Fi. Recharge Package

In [0]:
# It might be worthwhile to use the average recharge rate for the summer and a modified, higher recharge rate for non-summer.
# According to Burch, observed recharge is greatest following the end of the growing season, presumably in the fall, since precipitation available for recharge is limited to just after the growing season" (Burch, 2004).

# Average recharge = 333,000 gpd/sq mi = 0.001597 ft/d (Burch, 2004)
# For higher recharge rate, one option is to look at the water table fluctuation method and take the maximum recharge rate

# Recharge for cells where low-k material is at the surface is guessed to be an order of magnitude less than the recharge rate for cells with high-k material at the surface.
# The plan is to tinker with this assumption when we calibrate the model.

rch_avg=0.001597 # high recharge rate, in ft/d

# rch_low=rch_avg/10 # low recharge rate, in ft/d
# The model did not converge with this initial guess for the recharge rate for cells with low-k material at land surface

rch_low=rch_avg/2 # low recharge rate, in ft/d
# The model converged with this recharge rate, but I'm concerned it may be inaccurate

# Low-k material is at the surface in any cells where k=kf for layer #1 (kl1 = kf)
# High-k material is at the surface in any cells where k=kc for layer #1 (kl1 = kc)

# For cells where kl1 = kf, recharge is rch_low.
# For cells where kl1 = kc, recharge is rch_avg.
recharge=np.where(kl1<=kf,rch_low,rch_avg) # This line of code was made possible by the Will County group, and viewers like you!

# Assign recharge to the model
rch = flopy.modflow.mfrch.ModflowRch(model=m,nrchop=3,rech = recharge) # recharge rate is in ft/day

### 3Fii. River Package

In [0]:
# put into a format that MODFLOW wants
arriv = dfriv.values
riverdata = {0: arriv}

# create river package
riv = flopy.modflow.mfriv.ModflowRiv(model=m,ipakcb=None,stress_period_data=riverdata)

### 3Fiii. Drain Package


In [0]:
# Drain Package (DRN)
# Used to simulate head-dependent flux boundaries.  If the head in a cell falls below a certain threshold, the flux from the drain cell to the model cell drops to zero.

# Create an empty list as a starting point for our drain data
# Here, "ardrn" will function similarly to "arriv" in the previous code block
ardrn=[]

# "For" loop to search through the kl1 array and assign a drain conductance to cells where low-k material is at the surface
# Low-k material is at the surface in any cells where k=kf for the top layer (kl1=kf)
for row in range(m.nrow):
  for col in range(m.ncol):
    if round(float(kl1[row][col]),5)==kf: # This "if" statement covers cells where low-k material is present at land surface
      #C=kf*100*dx*dy/3 # C is the conductance between land surface and the drain, in ft^2/day.
          
      C=kf*10*dx*dy/3 # Tried reducing the conductance for the drain cells, to see if MODFLOW terminates normally.
      # This worked, but I'm concerned the conductance is inaccurate.
      
      # For more information regarding the equation for conductance, see the "notes on the conductance of drain cells" comment below.
      
      # Append the drain array "ardrn" with a list corresponding to each cell to be added to the drain package
      # Items in the drain array are lists with the format [Layer, Row, Column, Elevation, Conductance]
      ardrn.append([0, row, col, topgrid[row][col], C])

# Notes on the conductance of drain cells
# C=K*dx*dy/delta ; C=K*dx*dy/b
# K is the conductivity of the soil layer
# delta or b is the thickness of the soil layer
# Out of a handful of randomly selected well logs, the uppermost soil layer (where present) reached a depth anywhere from 1 to 5 feet.
# For simplicity, the soil layer is assumed to have a thickness of 3 feet, uniform throughout the model.

# From the FloPy documentation:
# flopy.modflow.mfdrn.ModflowDrn(model, ipakcb=None, stress_period_data=None, dtype=None, extension='drn', unitnumber=None, options=None, filenames=None, **kwargs)

In [0]:
# Drain Return Package (DRT)
# Allows simulation of drains in which some of the water from the drain is used to recharge the aquifer

# remove the drn package
#m.remove_package('DRN')

# create the drt package
#spd=[]
#for i in range(m.nrow):
    #spd.append([0, i, m.ncol-1, 5.0, 50.0, 1, 1, 1, 1.0])
#d = flopy.modflow.ModflowDrt(m, stress_period_data={0:spd})

In [0]:
# BEWARE: This cell alone takes about 3.5 minutes to run

# This cell ensures that no river cells are included in the drain package
# If any river cells were added to the drain array, this code block will remove them

ardrn_initial=len(ardrn)

# Print the initial length of the drain array
print('The initial length of the drain array is',ardrn_initial)

for i in range(len(dfriv)-1):
  for j in range(len(ardrn)-1):
    if dfriv.loc[i,'row']==np.float64(ardrn[j][1]) and dfriv.loc[i,'col']==np.float64(ardrn[j][2]):
      ardrn.remove(ardrn[j])

ardrn_final=len(ardrn)

# Print the final length of the drain array
# The drain array should be 1500-2000 cells shorter than it was before
print('The final length of the drain array is',ardrn_final)
print(ardrn_initial-ardrn_final,'items were removed')

In [0]:
# Put the drain data into a format that MODFLOW wants
draindata={0:ardrn} #Here, "draindata" functions similarly to "riverdata" in the previous code block

# Create the drn package
drn = flopy.modflow.mfdrn.ModflowDrn(m, ipakcb=None, stress_period_data=draindata)

##3Fiv. Well Package

In [0]:
#just some functions for processing json into model coordinates
#nothing to see here... move along

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def determine_layer(dtw, row, column, botgrids=botgrids, topgrid=topgrid):
  depth = topgrid[int(row), int(column)] - dtw
  if depth < botgrids[-1][int(row), int(column)]: return False
  for i, elem in enumerate(botgrids):
    nextelem = botgrids[(i + 1) % len(botgrids)][int(row), int(column)]
    if depth < elem[int(row), int(column)] and depth > nextelem:  return i

def convert_raw(well, maxrow=maxrow, maxcol=maxcol, ney=ney, swx=swx):
  row, column = np.trunc((ney-well[1])/dy), np.trunc((well[2]-swx)/dx)
  row -= 1
  column -= 1
  if row>=maxrow or row<0 or column>=maxcol or column<0:  return[False * 4]
  elif well[0] == -999:  return [6, int(row), int(column), well[3]]
  else: return [determine_layer(well[0], row, column), int(row), int(column), well[3]]

def convert_to_LatLong(row, illimap = illimap, latlong = wgs84):
  long, lat = pyproj.transform(illimap,latlong,row[2]/3.281, row[1]/3.281)
  return [row[0], lat, long, row[3]]

In [0]:
#!upload the processPumpData_SUMMER.json OR processPumpData_SPRING.json file from the ./pumping directory
#make sure you've created them by running processPumpData.py

#ensure the file you don't want to use is commented out
#future iterations of this model will utilize multiple stress periods, and will not require file switching
#to calibrate spring and summer pumpage, but client deadlines do not permit such major code refactoring
#at this time...

file_name = 'processPumpData_SUMMER.json'
#file_name = 'processPumpData_SPRING.json'

uploaded = files.upload() #Ah, I see you have the machine that goes 'ping!'.

#if you get an error when running this cell for the first time
#try rerunning this cell again

lrcq = json.loads(uploaded[file_name].decode("utf-8")) 
#This is my favourite. You see, we lease this back from the company we sold it to - 
#that way it comes under the monthly current budget and not the capital account.

lrcq = {int(k):v for k,v in lrcq.items()}
plot_points_df = pd.DataFrame(lrcq)

lrcq.update({0:[convert_raw(x) for x in lrcq[0] if convert_raw(x)[0]]})

wel = flopy.modflow.ModflowWel(model = m, stress_period_data=lrcq)

del uploaded
print("!WELLS SUCCESSFULLY ADDED TO MODEL!")

In [0]:
#map of pumpage data



fig=plt.figure(figsize=(10,10)) #create 10 x 10 figure
#Define a projection
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
#Define the spatial domain to plot
ax.set_xlim(sw_long,ne_long) #The spatial domain spans from minimum longitude to maximum longitude (x-direction)
ax.set_ylim(sw_lat,ne_lat) #The spatial domain spans from minimum latitude to maximum latitude (y-direction)
#Define a title for the plot
ax.set_title('Map of pumpage in region')

#Using CartoPy, import several features from Natural Earth Data (see https://www.naturalearthdata.com/features/)
#Large rivers:
largerivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines',
    scale='110m', # major rivers
    facecolor='none')
#Small rivers:
smallrivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines_scale_rank',
    scale='10m', # smaller rivers (still considered major by many/most people)
    facecolor='none')
#Smallest rivers:
smallestrivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_north_america',
    scale='10m',
    facecolor='none')
#Population centers:
popplaces = cf.NaturalEarthFeature(
    category='cultural',
    name='urban_areas', # plots municipal boundaries
    scale='10m',
    facecolor='plum')
#Major roads:
majorroads = cf.NaturalEarthFeature(
    category='cultural',
    name='roads',
    scale='10m',
    facecolor='none')

# Add the features defined above to the plot, with various colors and markers
#ax.add_feature(popplaces,edgecolor='plum',linewidth=1.0) # Population centers will be light purple areas
ax.add_feature(largerivers,edgecolor='aqua',linewidth=2.0) # Large, small, and smallest rivers will be light blue lines
ax.add_feature(smallrivers,edgecolor='aqua',linewidth=1.5)
ax.add_feature(smallestrivers,edgecolor='aqua',linewidth=1.0)
#ax.add_feature(majorroads,edgecolor='gray',linewidth=1.0) #Roads will be thin gray lines
ax.add_feature(USCOUNTIES.with_scale('5m'))

for row in plot_points_df.iterrows():
  ax.plot(convert_to_LatLong(row[1][0])[1], convert_to_LatLong(row[1][0])[2], marker='o', color = 'k', zorder = 10)

## 3G. Define output control

In [0]:
#create oc stress period data. 
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget', 'save drawdown']}
#create output control object
oc = flopy.modflow.ModflowOc(model=m, stress_period_data=spd, compact=True)

## 3H. Solver

In [0]:
# We will start by using the PCG solver with default settings
#pcg = flopy.modflow.ModflowPcg(model=m)
pcg = flopy.modflow.ModflowPcg(model=m,mxiter=200,iter1=50,hclose=1e-03,rclose=1e-03,relax=0.98,damp=0.3)

## 3I. Plot model inputs (boundary conditions, elevations)

In [0]:
'''Plot grid and boundary conditions [without drains]'''
#----------------------------------------------------------------------------
plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0)
#grid = modelmap.plot_grid()
ib = modelmap.plot_ibound()
rvr = modelmap.plot_bc(ftype='RIV')
wel = modelmap.plot_bc(ftype='WEL', plotAll = True)

#add labels and legend
plt.xlabel('Lx (ft)',fontsize = 14)
plt.ylabel('Ly (ft)',fontsize = 14)
plt.title('Boundary Conditions', fontsize = 15, fontweight = 'bold')
plt.legend(handles=[mp.patches.Patch(color='blue',label='Const. Head',ec='black'),
                    mp.patches.Patch(color='white',label='Active Cell',ec='black'),
                    mp.patches.Patch(color='black',label='Inactive Cell',ec='black'),
                    mp.patches.Patch(color='green',label='River',ec='green'),
                    mp.patches.Patch(color='red',label='Well',ec='red')],
                    bbox_to_anchor=(1.5,1.0))
plt.show()
#----------------------------------------------------------------------------

In [0]:
'''Plot Recharge'''
#----------------------------------------------------------------------------
plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.map.PlotMapView(model=m, layer=0) #use plotmapview to attach plot to model
#contour_levels = np.linspace(400,800,41)
#topelevations = modelmap.contour_array(topgrid, levels = contour_levels) #create head contours
#plt.clabel(topelevations, inline=True,fontsize=12,fmt='%1.0f')

#create colormap of named colors
colors = ["saddlebrown","lightgoldenrodyellow"]
cmap = mp.colors.LinearSegmentedColormap.from_list("", colors)
norm = mp.colors.LogNorm(vmin=kf,vmax=kc)
modelmap.plot_array(khlayers[0],norm = norm,cmap=cmap)
rvr = modelmap.plot_bc(ftype='RIV')
ib = modelmap.plot_ibound()
#display parameters
plt.xlabel('Lx (ft)',fontsize = 14)
plt.ylabel('Ly (ft)',fontsize = 14)
plt.title('Surficial Conductivity', fontsize = 15, fontweight = 'bold')
plt.legend(handles=[mp.patches.Patch(color='saddlebrown',label='Low Conductivity',ec='saddlebrown'),
                    mp.patches.Patch(color='white',label='Recharge=7.985E-4 ft/d',ec='white'),
                    mp.patches.Patch(color='white',ec='white'),
                    mp.patches.Patch(color='lightgoldenrodyellow',label='High Conductivity',ec='lightgoldenrodyellow'),
                    mp.patches.Patch(color='white',label='Recharge=1.597E-3 ft/d',ec='white')],
                    bbox_to_anchor=(1.5,1.0))
plt.show()

In [0]:
plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.map.PlotMapView(model=m, layer=0) #use plotmapview to attach plot to model
#contour_levels = np.linspace(400,800,41)
#topelevations = modelmap.contour_array(topgrid, levels = contour_levels) #create head contours
#plt.clabel(topelevations, inline=True,fontsize=12,fmt='%1.0f')

#create colormap  that works with both logarithmic scaling and normal scaling
norm = mp.colors.LogNorm(vmin=Tmin,vmax=Tmax)
#Plot options for normal/log plots
#x = modelmap.plot_array(Total, norm = norm, cmap= 'plasma')
x = modelmap.plot_array(Total, cmap= 'plasma')
# model bounds
rvr = modelmap.plot_bc(ftype='RIV')
ib = modelmap.plot_ibound()

#display parameters
plt.xlabel('Lx (ft)',fontsize = 14)
plt.ylabel('Ly (ft)',fontsize = 14)
plt.colorbar(x)
plt.title('Transmissivity [ft^2/day]', fontsize = 15, fontweight = 'bold')

In [0]:
'''Plot the boundary conditions'''
#----------------------------------------------------------------------------
# Be sure to verify that the drain cells work as intended:
# - Check that the locations of drain cells in the map below (magenta) match the locations of low-k material at land surface in the map above (dark brown)
# - Additionally, check that the river cells appear in the map below, to ensure that they were not overwritten as drain cells.

plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0)
#grid = modelmap.plot_grid()
ib = modelmap.plot_ibound()
rvr = modelmap.plot_bc(ftype='RIV')
drain = modelmap.plot_bc(ftype='DRN',color='magenta')
wel = modelmap.plot_bc(ftype='WEL', plotAll = True)

#add labels and legend
plt.xlabel('Lx (ft)',fontsize = 14)
plt.ylabel('Ly (ft)',fontsize = 14)
plt.title('Boundary Conditions', fontsize = 15, fontweight = 'bold')
plt.legend(handles=[mp.patches.Patch(color='green',label='River',ec='green'),
                    mp.patches.Patch(color='magenta',label='Drain',ec='magenta'),
                    mp.patches.Patch(color='red',label='Well',ec='red')],
                    bbox_to_anchor=(1.5,1.0))
plt.show()

In [0]:
np.array(khlayers).shape

In [0]:
#Plot representative cross section
plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelxsect = flopy.plot.PlotCrossSection(model = m, line={"row":50}) #use plotmapview to attach plot to model
#modelxsect = flopy.plot.PlotCrossSection(model = m, line={"column":100})
#create colormap of named colors
colors = ["saddlebrown","gray","lightgoldenrodyellow"]
cmap = mp.colors.LinearSegmentedColormap.from_list("", colors)
norm = mp.colors.LogNorm(vmin=kf,vmax=kc)
#modelxsect.plot_grid()
khlaynp = np.array(khlayers)
lines = modelxsect.plot_array(khlaynp,norm=norm, cmap=cmap)
rvr = modelxsect.plot_bc(ftype='RIV')
wel = modelxsect.plot_bc(ftype='WEL')
modelxsect.plot_ibound()

# 4 Write and run the MODFLOW model

In [0]:
# Write the model input
m.write_input()
# Execute the model run
success, mfoutput = m.run_model(pause=False, report=True)
# Report back if the model did not successfully complete
if not success:
    raise Exception('MODFLOW did not terminate normally.')

# 5 Plot Output Data

In [0]:
'''Extract binary data from head and flow files'''
#----------------------------------------------------------------------------
#extract binary data from head file as flopy head object
headobj = flopy.utils.binaryfile.HeadFile(modelname+'.hds')
#extract head data from head object
head = headobj.get_data(totim=1.0)

In [0]:
'''Plot results'''
#----------------------------------------------------------------------------
plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.map.PlotMapView(model=m, layer=0) #use plotmapview to attach plot to model
#grid = modelmap.plot_grid() #plot model grid
#contour_levels = np.linspace(head[0].min(),head[0].max(),11) #set contour levels for contouring head
contour_levels = np.linspace(200,1000,61) #set contour levels for contouring head
head_contours = modelmap.contour_array(head[8], levels=contour_levels) #create head contours
plt.clabel(head_contours, inline=True,fontsize=12,fmt='%1.0f')
rvr = modelmap.plot_bc(ftype='RIV')

#display parameters
plt.xlabel('Lx (ft)',fontsize = 14)
plt.ylabel('Ly (ft)',fontsize = 14)
plt.title('Steady-State Model, Flow(ft^3/d) and Head(ft) Results', fontsize = 15, fontweight = 'bold')
plt.show()

# 6 Model Calibration

In [0]:
# Import "GRL_basic_info.csv" and read in as a dataframe
# Work-around due to issues importing the files from GitHub
uploaded = files.upload()

del uploaded

In [0]:
# Import "import_spring2019.csv" and read in as a dataframe
# Work-around due to issues importing the files from GitHub
uploaded = files.upload()
df_spring2019 = pd.read_csv('import_spring2019.csv',index_col=0)

del uploaded

In [0]:
# Import "import_summer2019.csv" and read in as a dataframe
# Work-around due to issues importing the files from GitHub
uploaded = files.upload()
df_summer2019 = pd.read_csv('import_summer2019.csv',index_col=0)

del uploaded

In [0]:
# df_info

# Read in "GRL_basic_info.csv" as a dataframe
df_info = pd.read_csv('GRL_basic_info.csv',index_col=0)

# Drop the row corresponding to the well with P-number 381671
# This well has no measurements to which we can calibrate the model
df_info=df_info.drop([381671])

# Convert X_LAMBERT to column and Y_LAMBERT to row, and convert the new rows and columns to integers
df_info['row'] = np.trunc((ney-df_info['Y_LAMBERT'])/dy)
df_info['col'] = np.trunc((df_info['X_LAMBERT']-swx)/dx)
df_info = df_info.astype({'row': int,'col': int})

# trim the "df_info" dataframe to the model domain
df_info = df_info.loc[df_info['X_LAMBERT']<nex]
df_info = df_info.loc[df_info['Y_LAMBERT']<ney]
df_info = df_info.loc[df_info['X_LAMBERT']>swx]
df_info = df_info.loc[df_info['Y_LAMBERT']>swy]

# Add in a column to "df_info" that includes the head at each well, as calculated by the model
# These heads will be compared to the observed heads in the "df_waterlevels" dataframe
df_info['head_calculated_ft']=np.nan

# The following "for" loop fills in the "head_calculated_ft" column with the calculated head values from the model for each observation well
for value in df_info.index:
  # The following "if / else if" conditions identify the bottom-most layer of the model, for layers 4-9, where coarse material is located
  # In other words, the following lines of code identify the deepest layer, between layers 4 and 9, that corresponds to the Sankoty aquifer
  # The calculated heads for cells containing observation wells will be taken from this layer, in order to compare to the observed heads
  if df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl9[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 9
    df_info.at[value,'head_calculated_ft'] = head[8][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl8[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 8
    df_info.at[value,'head_calculated_ft'] = head[7][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl7[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 7
    df_info.at[value,'head_calculated_ft'] = head[6][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl6[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 6
    df_info.at[value,'head_calculated_ft'] = head[5][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl5[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 5
    df_info.at[value,'head_calculated_ft'] = head[4][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY' and round(float(kl4[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 4
    df_info.at[value,'head_calculated_ft'] = head[3][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='SANKOTY':  # Contingency case: if no cells are found that correspond to the Sankoty aquifer, layer 9 is used.
                                                        # This case signals a limitation of our model.  Specifically, our cross-sections do not capture the presence of coarse material corresponding to the Sankoty aquifer at the location of an observation well, where coarse material must exist.
    df_info.at[value,'head_calculated_ft'] = head[8][df_info.loc[value,'row']][df_info.loc[value,'col']]
  
  # The following "if / else if" conditions identify the bottom-most layer of the model, for layers 1-3, where coarse material is located
  # In other words, the following lines of code identify the deepest layer, between layers 1 and 3, that corresponds to the Tampico aquifer
  # The calculated heads for cells containing observation wells will be taken from this layer, in order to compare to the observed heads
  if df_info.loc[value,'LOCAL_AQ_NAME']=='TAMPICO' and round(float(kl3[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 3
    df_info.at[value,'head_calculated_ft'] = head[2][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='TAMPICO' and round(float(kl2[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 2
    df_info.at[value,'head_calculated_ft'] = head[1][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='TAMPICO' and round(float(kl1[df_info.loc[value,'row']][df_info.loc[value,'col']]),5)==kf: #Layer 1
    df_info.at[value,'head_calculated_ft'] = head[0][df_info.loc[value,'row']][df_info.loc[value,'col']]
  elif df_info.loc[value,'LOCAL_AQ_NAME']=='TAMPICO':  # Contingency case: if no cells are found that correspond to the Tampico aquifer, layer 3 is used.
                                                        # This case signals a limitation of our model.  Specifically, our cross-sections do not capture the presence of coarse material corresponding to the Tampico aquifer at the location of an observation well, where coarse material must exist.
    df_info.at[value,'head_calculated_ft'] = head[2][df_info.loc[value,'row']][df_info.loc[value,'col']]

### Calibration for Spring 2019

In [0]:
# Calibration for Spring 2019
# Make sure SPRING wells are used

# Create a new column in the "df_info" dataframe for observed heads from Summer 2019
df_info['head_spring2019_ft']=np.nan

# Change the NaN values in the new column to the observed head values, obtained from the imported "df_summer2019" dataframe
for value in df_info.index:
  df_info.at[value,'head_spring2019_ft'] = df_spring2019.loc[value,'Water_Surface_Elevation']

# Plot the calculated and observed values for comparison
plt.scatter(df_info.head_spring2019_ft, df_info.head_calculated_ft)
plt.plot(df_info.head_spring2019_ft,df_info.head_spring2019_ft)

one_to_one = mp.lines.Line2D(df_info.head_spring2019_ft,df_info.head_spring2019_ft, label='1 to 1 Line')

plt.xlabel('Observed Water Level (ft)')
plt.ylabel('Calculated Water Level (ft)')
plt.legend(handles=[one_to_one])
plt.show()

In [0]:
# Calculate the RMS error
df_info['diff^2_ft']=(df_info.head_calculated_ft-df_info.head_spring2019_ft)**2 # Create a new column in "df_info", containing the differences squared between the calculated and observed head values
RMS=math.sqrt(np.mean(df_info['diff^2_ft']))# Calculate the RMS error as the square root of the average of the new "diff^2_ft" column
print('The RMS error was calculated to be',RMS)

# Create a new column in the "df_info" dataframe containing the calculated error for each observation well
# Positive error means the model head is too low (not enough water)
# Negative error means the model head is too high (too much water)
df_info['error']=df_info['head_spring2019_ft']-df_info['head_calculated_ft']

### Calibration for Summer 2019

In [0]:
# Calibration for Summer 2019
# Make sure SUMMER wells are used

# Create a new column in the "df_info" dataframe for observed heads from Summer 2019
df_info['head_summer2019_ft']=np.nan

# Change the NaN values in the new column to the observed head values, obtained from the imported "df_summer2019" dataframe
for value in df_info.index:
  df_info.at[value,'head_summer2019_ft'] = df_summer2019.loc[value,'Water_Surface_Elevation']

# Plot the calculated and observed values for comparison
plt.scatter(df_info.head_summer2019_ft, df_info.head_calculated_ft)
plt.plot(df_info.head_summer2019_ft, df_info.head_summer2019_ft)

one_to_one = mp.lines.Line2D(df_info.head_summer2019_ft,df_info.head_summer2019_ft, label='1 to 1 Line')

plt.xlabel('Observed Water Level (ft)')
plt.ylabel('Calculated Water Level (ft)')
plt.legend(handles=[one_to_one])
plt.show()

In [0]:
# Calculate the RMS error
df_info['diff^2_ft']=(df_info.head_calculated_ft-df_info.head_summer2019_ft)**2 # Create a new column in "df_info", containing the differences squared between the calculated and observed head values
RMS=math.sqrt(np.mean(df_info['diff^2_ft']))# Calculate the RMS error as the square root of the average of the new "diff^2_ft" column
print('The RMS error was calculated to be',RMS)

# Create a new column in the "df_info" dataframe containing the calculated error for each observation well
# Positive error means the model head is too low (not enough water)
# Negative error means the model head is too high (too much water)
df_info['error']=df_info['head_summer2019_ft']-df_info['head_calculated_ft']

### Plot the calculated error on a map

In [0]:
# Note that the code blocks below will pull the data for whichever scenario was simulated, summer or spring

In [0]:
#Define the area over which to plot data

#Conduct the Universal Kriging, for the most recent data point
#The summer 2019 data will be addressed later
UK = UniversalKriging(df_info['LONG_WGS_84'], df_info['LAT_WGS_84'],df_info['error'], variogram_model='spherical',nlags=6)

#Create xpoints and ypoints in space, with 0.01 spacing
xpoints = np.arange(sw_long,ne_long,0.01)
ypoints = np.arange(sw_lat,ne_lat,0.01)

# create a meshgrid with xpoints and ypoints, to be used later in the code
X,Y = np.meshgrid(xpoints,ypoints)

# calculate the interpolated grid and fill values.
z, var = UK.execute('grid', xpoints, ypoints)
z = z.filled(fill_value=None)

In [0]:
#Create a figure
fig=plt.figure(figsize=(10,10)) #create 10 x 10 figure
#Define a projection
ax = plt.axes(projection=ccrs.PlateCarree())
#Define the spatial domain to plot
ax.set_xlim(sw_long,ne_long) #The spatial domain spans from minimum longitude to maximum longitude (x-direction)
ax.set_ylim(sw_lat,ne_lat) #The spatial domain spans from minimum longitude to maximum latitude (y-direction)
#Define a title for the plot
ax.set_title('Calculated error (ft difference) over the model area')

#Using CartoPy, import several features from Natural Earth Data (see https://www.naturalearthdata.com/features/)
#Large rivers:
largerivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines',
    scale='110m', # major rivers
    facecolor='none')
#Small rivers:
smallrivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines_scale_rank',
    scale='10m', # smaller rivers (still considered major by many/most people)
    facecolor='none')
#Smallest rivers:
smallestrivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_north_america',
    scale='10m',
    facecolor='none')
#Population centers:
popplaces = cf.NaturalEarthFeature(
    category='cultural',
    name='urban_areas', # plots municipal boundaries
    scale='10m',
    facecolor='plum')
#Major roads:
majorroads = cf.NaturalEarthFeature(
    category='cultural',
    name='roads',
    scale='10m',
    facecolor='none')

# Add the features defined above to the plot, with various colors and markers
ax.add_feature(popplaces,edgecolor='plum',linewidth=1.0) # Population centers will be light purple areas
ax.add_feature(largerivers,edgecolor='aqua',linewidth=2.0) # Large, small, and smallest rivers will be light blue lines
ax.add_feature(smallrivers,edgecolor='aqua',linewidth=2.0)
ax.add_feature(smallestrivers,edgecolor='aqua',linewidth=2.0)
ax.add_feature(majorroads,edgecolor='gray',linewidth=1.0) #Roads will be thin gray lines
ax.add_feature(USCOUNTIES.with_scale('5m'))

#Create contours from the interpolation
error_contour_levels = [-60,-45,-30,-15,0,15,30,45,60,75] # Set contour levels for contouring error, use this line for summer
#error_contour_levels = [-80,-70,-60,-50,-40,-30,-20,-10,0,10] # Set contour levels for contouring error, use this line for spring
cset_contour = plt.contour(X,Y,z,error_contour_levels,colors='blue')

#Add in a continuous color flood
#cset_fill=plt.imshow(z,vmin=-100,vmax=100,cmap=plt.cm.coolwarm,origin='lower',extent=[X.min(), X.max(), Y.min(), Y.max()])
cset_fill=plt.imshow(z,vmin=-100,vmax=100,cmap=plt.cm.coolwarm,origin='lower',extent=[X.min(), X.max(), Y.min(), Y.max()])

#The following code could also be used, if a discrete color flood is preferred:
#cset_fill = plt.contourf(X,Y,z,vmin=-100,vmax=100,cmap=plt.cm.coolwarm)

#Label contours (makes use of pylab)
pylab.clabel(cset_contour, inline=1, fontsize=10,fmt='%1.0f')

#Plot the points that were measured
points=plt.scatter(df_info['LONG_WGS_84'], df_info['LAT_WGS_84'], marker=".", color="black", label="Data Points")

#Add a legend to the plot
cities_towns=mpatches.Patch(color='plum',label='Population Centers') #Here I create a proxy artist to represent population centers on the legend
rivers=mlines.Line2D([],[],linewidth=2.0,color='aqua',label='Rivers') #Proxy artist for large and small rivers
roads=mlines.Line2D([],[],linewidth=1.0,color='gray',label='Roads') #Proxy artist for major roads
ax.legend(loc='lower left',handles=[cities_towns,points,rivers,roads])

#Add in a colorbar
cax = fig.add_axes([0.34, 0.32, 0.25, 0.03])
fig.colorbar(cset_fill, cax=cax, orientation='horizontal')
plt.show()

In [0]:
df_info.error.min()

In [0]:
df_info.error.max()

# 7 Calculate Discharge

In [0]:
# first of all, read my_model.list file (this contains the information taht we need for discharge)
# the budget is returned as recarray
from flopy.utils import mflistfile
mf_list = mflistfile.MfusgListBudget("my_model.list")
budget = mf_list.get_cumulative()

In [0]:
# convert the recarray to pandas dataframe
data = pd.DataFrame(budget)
# print(data)

In [0]:
# use the river leakage out minus in to calculate the discharge at current situation (summer 2019)
discharge_current = int(data['RIVER_LEAKAGE_OUT'] - data['RIVER_LEAKAGE_IN'])  #ft3/day
print('The discharge of current situation is', discharge_current, 'ft3/day.')

In [0]:
# discharge_pre is the discharge of pre-development where there's no pumping
# I use the same method as above to calculate this at a different file and copy paste the result here
discharge_pre = 38593780 #ft3/day
discharge_diff = discharge_pre - discharge_current
print('The reduction in natural groundwater discharge is', discharge_diff, 'ft3/day.')